#### Canada regions dataset

In [1]:
import geopandas as gpd
import requests

url = 'https://opendata.arcgis.com/datasets/e5403793c5654affac0942432783365a_0.geojson'
r = requests.get(url)

with open('../../Data/app updated data/Health_Region_Summaries.geojson', 'wb') as f:
    f.write(r.content)

summary_canada = gpd.read_file('../../Data/app updated data/Health_Region_Summaries.geojson')

In [2]:
summary_canada.head(2)

,OBJECTID,HR_UID,Province,ENGNAME,FRENAME,CaseCount,Deaths,Recovered,Tests,TotalPop2019,...,Pop80to84_2019,Pop85Older,AverageAge_2019,MedianAge_2019,Last_Updated,SourceURL,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,3537,ON,City of Hamilton Health Unit,Circonscription sanitaire de la cité de Hamilton,861,44.0,790.0,43459.0,576272.0,...,12640.0,14790.0,41.4,40.5,2020-07-12T00:43:59,https://www.hamilton.ca/coronavirus/status-cases,{9217ED6B-3462-4814-8954-C6F9E2A98495},1.830824,0.127329,"MULTIPOLYGON (((-79.79317 43.30113, -79.79296 ..."
1,2,3538,ON,Hastings and Prince Edward Counties Health Unit,Circonscription sanitaire des comtés de Hastin...,43,5.0,37.0,13110.0,144779.0,...,4381.0,4215.0,45.3,48.3,2020-07-11T16:15:59,https://hpepublichealth.ca/covid-19-cases/,{E7C3696B-1EF1-45BE-87DC-C0F6CF8B48CD},7.844390,0.842562,"MULTIPOLYGON (((-77.50018 43.91084, -77.51734 ..."


- Ontario

In [3]:
def Ontario_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_ON = summary_canada.loc[summary_canada['Province'] == 'ON'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    ON_regions = gpd.read_file('../../Data/app updated data/ON2.shp')
    for row_num in range(len(ON_regions)):
        Region = ON_regions.loc[row_num, 'region'].replace('?','–')
        Region_value = summary_canada_ON.loc[summary_canada_ON['ENGNAME'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            ON_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))
    
    return(ON_regions)

In [4]:
ON_regions = Ontario_regions(summary_canada)
ON_regions.head()

The region Oxford Elgin St. Thomas Health Unit doesn't have any cases


,region,num cases,geometry
0,City of Hamilton Health Unit,861,"POLYGON ((-79.79317 43.30113, -79.77674 43.275..."
1,Hastings and Prince Edward Counties Health Unit,43,"MULTIPOLYGON (((-76.97240 43.98457, -76.98798 ..."
2,Huron Perth Public Health Unit,58,"POLYGON ((-80.99179 43.98641, -80.95765 43.840..."
3,Chatham-Kent Health Unit,166,"POLYGON ((-81.83506 42.65096, -81.83467 42.608..."
4,"Kingston, Frontenac and Lennox and Addington H...",105,"MULTIPOLYGON (((-77.00344 44.04905, -77.04079 ..."


- Quebec

In [5]:
def Quebec_tracker():
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests
    import unicodedata
    import geopandas as gpd
    from unidecode import unidecode
    url = 'https://www.quebec.ca/en/health/health-issues/a-z/2019-coronavirus/situation-coronavirus-in-quebec/'

    resp = requests.get(url)

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.content, 'html.parser')
        
        table_data = soup.find_all("div", {'class':'tableauOverflow'})
        ### Header
        header_values = [unicodedata.normalize("NFKD", i.text) for i in table_data[0].find_all("thead")[0].find_all("th")]
        Date = header_values[-1].split('on')[-1].strip().rstrip()
        ### Table
        Region_df = pd.DataFrame(columns = header_values)
        Region_df[header_values[0]] = [i.find_all("td")[0].text.replace('\xa0',' ') for i in table_data[0].find_all("tbody")[0].find_all("tr")]
        Region_df[header_values[1]] = [int(i.find_all("td")[1].text.replace('\xa0','').replace(' ','').replace('*','')) for i in table_data[0].find_all("tbody")[0].find_all("tr")]
        
        for row_num in range(len(Region_df)):
            if '-' in Region_df.loc[row_num, 'Regions'] and len(Region_df.loc[row_num, 'Regions'].split(' - ')[0]) == 2:
                Region_df.loc[row_num, 'Regions num'] = Region_df.loc[row_num, 'Regions'].split(' - ')[0]
            elif '–' in Region_df.loc[row_num, 'Regions'] and len(Region_df.loc[row_num, 'Regions'].split(' – ')[0]) == 2:
                Region_df.loc[row_num, 'Regions num'] = Region_df.loc[row_num, 'Regions'].split(' – ')[0]
            else:
                Region_df.loc[row_num, 'Regions num'] = '-'
        Regions_shp = gpd.read_file('../../Data/app updated data/QC 4326.shp')
        Regions_shp = Regions_shp.rename(columns = {'Regions nu':'Regions num'})
        Regions_shp_cases = pd.merge(Region_df , Regions_shp, on ='Regions num', how = 'inner')
        print('Missing QC region is:   {}'
              .format(','.join([i for i in Region_df['Regions num'].tolist() if i not in Regions_shp_cases['Regions num'].tolist()])))
        
        Regions_shp_cases['Regions'] = Regions_shp_cases['Regions'].apply(lambda x: unidecode(x))
        ## Writing the result
        #Regions_shp_cases.to_excel('../../Dash_app2/data/quebec/quebec_regions_report.xlsx', index = False)
        QC_df = pd.read_excel('../../Data/app updated data/QC result.xlsx')
        QC_df.loc[QC_df['Metric'] == 'Confirmed', 'Value'] = Region_df.loc[Region_df['Regions'] == 'Total', Region_df.columns[1]].values[0]
        QC_df.loc[QC_df['Metric'] == 'Date', 'Value'] = header_values[-1].split('on ')[-1]
        QC_df.loc[QC_df['Metric'] == 'Death', 'Value'] = int(soup.find_all("div", {'id':'c47903'})[0].text.split('deaths')[0].replace('\xa0','').rstrip().split(' ')[-1])
        #QC_df.to_excel('../../Dash_app2/data/quebec/QC result.xlsx', index = False)
        
        Regions_shp_cases = Regions_shp_cases.drop(columns = {'Regions num':'num cases'}, inplace = True)
        Regions_shp_cases = Regions_shp_cases.rename(columns = {'Regions':'region', Regions_shp_cases.columns[1]: 'num cases'})
        return(Regions_shp_cases, QC_df)
    else:
        print('Proxy issue.')

In [6]:
def Quebec_tracker(summary_canada):
    import unidecode
    import re
    import pandas as pd
    import geopandas as gpd
    
    summary_canada_QC = summary_canada.loc[summary_canada['Province'] == 'QC'].reset_index(drop = True)\
                                    .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    
    pattern = re.compile('[^a-zA-Z]+')
    locations_EN = [pattern.sub(' ', unidecode.unidecode(i)).strip().rstrip() for i in summary_canada_QC['ENGNAME'].tolist()]
    summary_canada_QC['region'] = locations_EN
    
    QC_Regions_shp = gpd.read_file('../../Data/app updated data/QC 4326 2.shp')
    QC_Regions_shp2 = pd.merge(QC_Regions_shp, summary_canada_QC[['region', 'CaseCount']], how = 'left', on = 'region')

    print("The region {} doesn't have any cases".format(QC_Regions_shp2.loc[QC_Regions_shp2['CaseCount'].isnull(), 'region']))
    QC_Regions_shp2.rename(columns = {'CaseCount':'num cases'}, inplace = True)
    return(QC_Regions_shp2, summary_canada_QC)

In [7]:
QC_regions, QC_df = Quebec_tracker(summary_canada)
QC_regions.head()

The region Series([], Name: region, dtype: object) doesn't have any cases


,region,geometry,num cases
0,Region du Bas Saint Laurent,"POLYGON ((-67.59801 47.99982, -68.12259 47.999...",62
1,Region du Saguenay Lac Saint Jean,"POLYGON ((-69.87082 48.18161, -69.95675 48.057...",342
2,Region de la Capitale Nationale,"POLYGON ((-69.53634 48.05613, -69.61044 48.000...",1874
3,Region de la Mauricie et du Centre du Quebec,"POLYGON ((-74.44389 48.92928, -73.72845 48.470...",2053
4,Region de l Estrie,"POLYGON ((-70.41383 45.79610, -70.40022 45.720...",979


- Alberta

In [8]:
def Alberta_regions_tracker():
    from shapely.geometry import Polygon
    from bs4 import BeautifulSoup
    import requests
    import geopandas as gpd
    import re
    url = 'https://covid19stats.alberta.ca/'

    resp = requests.get(url)

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.content, 'html.parser')
        ### Extracting lat long
        if '"method":"addPolygons",' in soup.text:
            all_lat_lng = soup.text.split('"method":"addPolygons",')[1].split('"args":')[1].replace('\r','').replace('\n','').split(',null,null,')[0]
            all_lat_lng_filtered = re.sub(' +', ' ',all_lat_lng.strip().rstrip())
            lngs = [i.split('],"lat"')[0] for i in all_lat_lng_filtered.split('[[{"lng":[')[1:]]
            lats = [i.split('],"lat":[')[1] for i in all_lat_lng_filtered.split(']}]],[[{"lng":[')]
            lats = [i.split(']}')[0] for i in lats]

            region_lat_lng = {}
            for row_num in range(len(lats)):
                region_lat_lng[row_num] = {
                    'lng':[float(i) for i in lngs[row_num].split(',')],\
                    'lat': [float(i) for i in lats[row_num].split(',')]}
            polygon = {}
            for i in range(len(region_lat_lng)):
                polygon[i] = Polygon(zip(region_lat_lng[i]['lng'], region_lat_lng[i]['lat']))
            ### Creating geo-pandas df
            crs = {'init': 'epsg:4326'}
            polygon_df = gpd.GeoDataFrame(index=list(polygon.keys()), crs=crs, geometry=list(polygon.values())) 
            ### Adding Region name and number
            regions_name_num = soup.text.split(',null,null,["')[1].split('],')[0]
            regions_name = [i.split('<\\/')[0] for i in regions_name_num.split('<strong>')[1:]]
            regions_num = [int(i.split(' Cases')[0].split('>')[-1].rstrip().strip()) for i in regions_name_num.split('<strong>')[1:]]
        
        elif '"method": "addPolygons",' in soup.text:
            all_lat_lng = soup.text.split('"method": "addPolygons",')[1].split('"args": ')[1].replace('\r','').replace('\n','').split(', null, null,')[0]
            all_lat_lng_filtered = re.sub(' +', ' ',all_lat_lng.strip().rstrip())
            lngs = [i.split('], "lat"')[0] for i in all_lat_lng_filtered.split('[ [{ "lng": [')[1:]]
            lats = [i.split('], "lat": [')[1] for i in all_lat_lng_filtered.split('] }] ], [ [{ "lng": [')]
            lats = [i.split('] }')[0] for i in lats]

            region_lat_lng = {}
            for row_num in range(len(lats)):
                region_lat_lng[row_num] = {
                    'lng':[float(i) for i in lngs[row_num].split(',')],\
                    'lat': [float(i) for i in lats[row_num].split(',')]}
        
            polygon = {}
            for i in range(len(region_lat_lng)):
                polygon[i] = Polygon(zip(region_lat_lng[i]['lng'], region_lat_lng[i]['lat']))
            ### Creating geo-pandas df
            crs = {'init': 'epsg:4326'}
            polygon_df = gpd.GeoDataFrame(index=list(polygon.keys()), crs=crs, geometry=list(polygon.values())) 
            ### Adding Region name and number
            regions_name_num = soup.text.split('null, null, ["')[1].split('],')[0]
            regions_name = [i.split('<\\/')[0] for i in regions_name_num.split('<strong>')[1:]]
            regions_num = [int(i.split(' Cases')[0].split('>')[-1].rstrip().strip()) for i in regions_name_num.split('<strong>')[1:]]
        
        else:
            print("Scraper couldn't find lat long s")
            
        polygon_df['region'] = regions_name[:-1]
        polygon_df['num cases'] = regions_num[:-1]
        polygon_df2 = polygon_df.loc[polygon_df['num cases']>0].reset_index(drop = True)
        #polygon_df2.to_file('../../Dash_app2/data/alberta/Alberta regions.json', driver='GeoJSON')
        #polygon_df2.to_file('../../Dash_app2/data/alberta/Alberta regions.shp')
        return(polygon_df2)

In [9]:
Alberta_regions = Alberta_regions_tracker()
Alberta_regions.head()

,geometry,region,num cases
0,"POLYGON ((-114.08376 51.29993, -114.07216 51.3...",City Of Airdrie,50
1,"POLYGON ((-113.36156 55.02383, -113.05654 55.0...",Athabasca County,2
2,"POLYGON ((-114.34920 54.50138, -114.30450 54.5...",County Of Barrhead No. 11,2
3,"POLYGON ((-113.46430 53.36665, -113.44263 53.3...",City Of Beaumont,9
4,"POLYGON ((-113.05000 53.35949, -113.03879 53.3...",Beaver County,5


- British Columbia

In [10]:
def BC_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_BC = summary_canada.loc[summary_canada['Province'] == 'BC'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    summary_canada_BC['region'] = summary_canada_BC['ENGNAME'].apply(lambda x : x.replace(' Health',''))
    summary_canada_BC.loc[summary_canada_BC['region'] == 'Vancouver  Coastal', 'region'] = 'Vancouver Coastal'
    
    BC_regions = gpd.read_file('../../Data/app updated data/BC new.shp')
    BC_regions.loc[BC_regions['region'] == 'Island', 'region'] = 'Vancouver Island'
    
    for row_num in range(len(BC_regions)):
        Region = BC_regions.loc[row_num, 'region']
        Region_value = summary_canada_BC.loc[summary_canada_BC['region'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            BC_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))
    
    return(BC_regions)

In [11]:
BC_regions = BC_regions(summary_canada)
BC_regions

,regions,region,geometry,num cases
0,Interior,Interior,"POLYGON ((-118.22435 52.37973, -118.18686 52.3...",202.0
1,Fraser,Fraser,"POLYGON ((-122.52210 50.12394, -122.25594 50.1...",1610.0
2,Vancouver Coastal,Vancouver Coastal,"MULTIPOLYGON (((-123.02309 49.29350, -123.0233...",990.0
3,Island,Vancouver Island,"MULTIPOLYGON (((-123.30838 48.82192, -123.2546...",135.0
4,Northern,Northern,"MULTIPOLYGON (((-131.09724 52.15232, -130.9926...",65.0


- Manitoba

In [12]:
def Manitoba_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_Manitoba = summary_canada.loc[summary_canada['Province'] == 'MB'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    
    Manitoba_regions = gpd.read_file('../../Data/app updated data/MN 4326.shp')
    Manitoba_regions.rename(columns = {'Region':'region'}, inplace = True)
    Manitoba_regions.loc[Manitoba_regions['region'] == 'Northern', 'region'] = 'Northern Regional Health Authority'
    Manitoba_regions.loc[Manitoba_regions['region'] == 'Southern', 'region'] = 'Southern Health—Santé Sud'
    Manitoba_regions.loc[Manitoba_regions['region'] == 'Winnipeg', 'region'] = 'Winnipeg Regional Health Authority'
    Manitoba_regions.loc[Manitoba_regions['region'] == 'Prairie Mountain', 'region'] = 'Prairie Mountain Health'
    Manitoba_regions.loc[Manitoba_regions['region'] == 'Interlake-Eastern', 'region'] = 'Interlake-Eastern Regional Health Authority'
    
    for row_num in range(len(Manitoba_regions)):
        Region = Manitoba_regions.loc[row_num, 'region']
        Region_value = summary_canada_Manitoba.loc[summary_canada_Manitoba['ENGNAME'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            Manitoba_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))
    
    return(Manitoba_regions)

In [13]:
Manitoba_regions = Manitoba_regions(summary_canada)
Manitoba_regions

,region,geometry,num cases
0,Northern Regional Health Authority,"POLYGON ((-94.20389 58.79421, -94.20374 58.784...",3.0
1,Southern Health—Santé Sud,"POLYGON ((-97.32040 49.92726, -97.32195 49.898...",40.0
2,Winnipeg Regional Health Authority,"MULTIPOLYGON (((-97.02616 49.79758, -97.04858 ...",236.0
3,Prairie Mountain Health,"POLYGON ((-98.34017 50.78035, -98.73817 50.772...",26.0
4,Interlake-Eastern Regional Health Authority,"POLYGON ((-94.89835 52.99758, -95.00359 52.932...",20.0


Saskatchewan

In [14]:
def Saskatchewan_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_Saskatchewan = summary_canada.loc[summary_canada['Province'] == 'SK'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    
    Saskatchewan_regions = gpd.read_file('../../Data/app updated data/Sask 2 4326.shp')
    Saskatchewan_regions.rename(columns = {'Region':'region'}, inplace = True)

    for row_num in range(len(Saskatchewan_regions)):
        Region = Saskatchewan_regions.loc[row_num, 'region']
        Region_value = summary_canada_Saskatchewan.loc[summary_canada_Saskatchewan['ENGNAME'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            Saskatchewan_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))
    Saskatchewan_regions.loc[:, 'region'] = Saskatchewan_regions.loc[:, 'region'].apply(lambda x : x + ' Saskatchewan' if x not in ['Regina', 'Saskatoon'] else x)
    
    return(Saskatchewan_regions)

In [15]:
Saskatchewan_regions = Saskatchewan_regions(summary_canada)
Saskatchewan_regions

,region,geometry,num cases
0,Far North Saskatchewan,"POLYGON ((-101.81044 54.15324, -102.04334 54.1...",339.0
1,North Saskatchewan,"POLYGON ((-101.63781 52.31815, -101.67045 52.3...",120.0
2,Central Saskatchewan,"MULTIPOLYGON (((-101.63781 52.31815, -101.6090...",16.0
3,Regina,"POLYGON ((-103.70429 51.50508, -103.70411 51.2...",80.0
4,Saskatoon,"POLYGON ((-103.28227 51.88146, -103.28232 51.7...",192.0
5,South Saskatchewan,"POLYGON ((-101.44729 49.97504, -101.44722 49.8...",68.0


- New Brunswick

In [16]:
def New_Brunswick_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_New_Brunswick = summary_canada.loc[summary_canada['Province'] == 'NB'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    
    New_Brunswick_regions = gpd.read_file('../../Data/app updated data/NB 4326.shp')
    New_Brunswick_regions.rename(columns = {'Region':'region'}, inplace = True)

    for row_num in range(len(New_Brunswick_regions)):
        Region_tmp = New_Brunswick_regions.loc[row_num, 'region']
        Region = [i for i in summary_canada_New_Brunswick['ENGNAME'].tolist() if Region_tmp in i][0]
        Region_value = summary_canada_New_Brunswick.loc[summary_canada_New_Brunswick['ENGNAME'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            New_Brunswick_regions.loc[row_num, 'num cases'] = Region_value.values[0]
            New_Brunswick_regions.loc[row_num, 'region'] = Region
        else:
            print("The region {} doesn't have any cases".format(Region))

    return(New_Brunswick_regions)

In [17]:
New_Brunswick_regions = New_Brunswick_regions(summary_canada)
New_Brunswick_regions

,region,num cases,geometry
0,Zone 7 (Miramichi area),2,"POLYGON ((-64.77856 47.08268, -64.80152 47.066..."
1,Zone 1 (Moncton area),27,"POLYGON ((-65.07681 45.51329, -65.11935 45.615..."
2,Zone 2 (Saint John area),27,"POLYGON ((-65.07681 45.51329, -65.22965 45.477..."
3,Zone 3 (Fredericton area),49,"POLYGON ((-67.17062 47.40173, -67.04824 47.362..."
4,Zone 4 (Edmundston area),7,"POLYGON ((-67.33959 47.86959, -67.32162 47.821..."
5,Zone 5 (Campbellton area),53,"POLYGON ((-65.72934 48.00003, -65.80742 47.855..."
6,Zone 6 (Bathurst area),1,"POLYGON ((-64.83284 47.45968, -64.93443 47.436..."


- Newfoundland and Labrador

In [18]:
def NL_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_NL = summary_canada.loc[summary_canada['Province'] == 'NL'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    summary_canada_NL['region'] = summary_canada_NL['ENGNAME'].apply(lambda x : x.replace('Regional Health Authority', 'Health'))
    
    NL_regions = gpd.read_file('../../Data/app updated data/New 4326.shp')
    NL_regions.rename(columns = {'Region':'region'}, inplace = True)

    for row_num in range(len(NL_regions)):
        Region = NL_regions.loc[row_num, 'region']
        Region_value = summary_canada_NL.loc[summary_canada_NL['region'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            NL_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))

    return(NL_regions)

In [19]:
NL_regions = NL_regions(summary_canada)
NL_regions

,region,num cases,geometry
0,Eastern Health,244,"POLYGON ((-55.99792 47.21657, -55.79226 47.221..."
1,Central Health,8,"POLYGON ((-53.51792 48.66549, -53.57259 48.578..."
2,Western Health,4,"POLYGON ((-56.43805 50.11598, -56.66050 49.827..."
3,Labrador-Grenfell Health,6,"POLYGON ((-55.50000 50.53765, -56.07399 50.395..."


- Nova Scotia

In [20]:
def Nova_Scotia_regions(summary_canada):
    import geopandas as gpd
    
    summary_canada_Nova_Scotia = summary_canada.loc[summary_canada['Province'] == 'NS'].reset_index(drop = True)\
                                        .loc[:,['ENGNAME', 'CaseCount', 'geometry']]
    summary_canada_Nova_Scotia['region'] = summary_canada_Nova_Scotia['ENGNAME'].apply(lambda x : x.split('- ')[1])
    
    Nova_Scotia_regions = gpd.read_file('../../Data/app updated data/NC 4326.shp')
    Nova_Scotia_regions['region'] = Nova_Scotia_regions['region'].apply(lambda x : x.replace('NC', 'Nova Scotia'))
    
    for row_num in range(len(Nova_Scotia_regions)):
        Region_tmp = Nova_Scotia_regions.loc[row_num, 'region']
        Region = [i for i in summary_canada_Nova_Scotia['region'].tolist() if i in Region_tmp][0]
        Region_value = summary_canada_Nova_Scotia.loc[summary_canada_Nova_Scotia['region'] == Region, 'CaseCount']
        if len(Region_value) == 1:
            Nova_Scotia_regions.loc[row_num, 'num cases'] = Region_value.values[0]
        else:
            print("The region {} doesn't have any cases".format(Region))

    return(Nova_Scotia_regions)

In [21]:
Nova_Scotia_regions = Nova_Scotia_regions(summary_canada)
Nova_Scotia_regions

,region,geometry,num cases
0,Eastern Nova Scotia,"POLYGON ((-62.07318 44.87949, -62.14707 44.946...",54.0
1,Central Nova Scotia,"MULTIPOLYGON (((-62.07318 44.87949, -62.08482 ...",901.0
2,Western Nova Scotia,"POLYGON ((-63.96423 44.50356, -64.00000 44.500...",54.0
3,Northern Nova Scotia,"POLYGON ((-62.24787 45.74941, -62.22412 45.696...",57.0


- Montreal

In [22]:
def Montreal_borough():
    import pandas as pd
    from bs4 import BeautifulSoup
    import requests
    import geopandas as gpd
    import unidecode

    #url = 'https://santemontreal.qc.ca/en/public/coronavirus-covid-19/#c36391'
    url = 'https://santemontreal.qc.ca/en/public/coronavirus-covid-19/situation-of-the-coronavirus-covid-19-in-montreal/#c41383'
    resp = requests.get(url)

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.content, 'html.parser')
        tables_data = soup.find_all("table", {'class':'contenttable'})
        if len (tables_data) > 1:
            table_data = tables_data[2]
            header_data = [i.text.replace('\xa0',' ') for i in table_data.find_all("thead")[0].find_all("tr")[0].find_all("th")]
            Montreal_bor_df = pd.DataFrame(columns = header_data)
            for col_num in range(len(header_data)):
                Montreal_bor_df.iloc[:,col_num] = [unidecode.unidecode(i.find_all("td")[col_num].text.replace("'",'').replace('\xa0',' ').replace('–','-').replace(',','').strip().rstrip()) for i in table_data.find_all("tbody")[0].find_all("tr")]
            Montreal_bor_df.rename(columns = {Montreal_bor_df.columns[0]:'Region'},inplace = True)

            ## Mapping with shapefile
            Montreal_bor_gpd = gpd.read_file('../../Data/app updated data/montreal_borough.shp')
            Montreal_bor_df2 = pd.merge(Montreal_bor_gpd, Montreal_bor_df, on = 'Region', how = 'left')
            print("These regions didn't matched\n {}"
              .format(','.join([i for i in list(set(Montreal_bor_df['Region'])) if i not in list(set(Montreal_bor_df2['Region']))])))

            Montreal_bor_df3 = Montreal_bor_df2.iloc[:,:3]
            num_cases_column = Montreal_bor_df3.columns.tolist()[-1]
            Montreal_bor_df3.loc[Montreal_bor_df3[num_cases_column] == '<5', num_cases_column] = 4
            Montreal_bor_df3.loc[Montreal_bor_df3[num_cases_column] == '< 5', num_cases_column] = 4
            Montreal_bor_df3 = Montreal_bor_df3.fillna(0)
            Montreal_bor_df3[num_cases_column] = Montreal_bor_df3[num_cases_column].apply(lambda x : int(str(x).replace(' ','')))
            Montreal_bor_df3.rename(columns = {'Region':'region', Montreal_bor_df3.columns.tolist()[-1]: 'num cases'}, inplace = True)
            return(Montreal_bor_df3)

In [23]:
Montreal_bor_df = Montreal_borough()
Montreal_bor_df.head()

These regions didn't matched
 Territory to be confirmed2,Total for Montreal


,region,geometry,num cases
0,Outremont,"POLYGON ((-73.62078 45.52365, -73.61723 45.527...",272
1,LaSalle,"POLYGON ((-73.66610 45.42180, -73.66587 45.432...",1213
2,Mont-Royal,"POLYGON ((-73.65075 45.52631, -73.64819 45.530...",282
3,Ville-Marie,"POLYGON ((-73.53013 45.53476, -73.53023 45.532...",819
4,Plateau-Mont-Royal,"POLYGON ((-73.55923 45.53990, -73.56138 45.535...",898


- Merging All the Regions to create 1 master shapefile

In [24]:
def CA_Provicnes(ON_regions, QC_regions, Alberta_regions, BC_regions, Manitoba_regions, Saskatchewan_regions,
                New_Brunswick_regions, NL_regions, Nova_Scotia_regions, Montreal_bor_df):
    import pandas as pd
    import geopandas as gpd
    from unidecode import unidecode
    import re
    import pickle
    import json
    
    Alberta_regions['Province'] = 'Alberta'
    QC_regions['Province'] = 'Quebec'
    BC_regions['Province'] = 'British Columbia'
    ON_regions['Province'] = 'Ontario'
    Manitoba_regions['Province'] = 'Manitoba'
    Saskatchewan_regions['Province'] = 'Saskatchewan'
    New_Brunswick_regions['Province'] = 'New Brunswick'
    NL_regions['Province'] = 'Newfoundland and Labrador'
    Montreal_bor_df['Province'] = 'Montreal'
    Nova_Scotia_regions['Province'] = 'Nova Scotia'

    CA_Provinces_result = pd.concat([Alberta_regions, QC_regions, BC_regions, ON_regions, Manitoba_regions, 
                                     Saskatchewan_regions, New_Brunswick_regions, NL_regions, Montreal_bor_df, 
                                     Nova_Scotia_regions], 
                                    ignore_index = True, sort = False)
    CA_Provinces_result['region'] = CA_Provinces_result['region'].apply(lambda x : re.sub('\W+', ' ', unidecode(x)))
    CA_Provinces_result = CA_Provinces_result[['geometry','region','num cases','Province']]
    #CA_Provinces_result.drop(columns = 'geometry').to_excel('../../Dash_app2/data/CA Provinces.xlsx', index = False)
    
    CA_Provinces_result1 = pd.DataFrame(columns = ['region','num cases','Province'])
    for col in CA_Provinces_result1.columns:
        CA_Provinces_result1[col] = CA_Provinces_result[col]
    with open('../../Dash_app2/data/Canada regions Table.pickle', 'wb') as handle:
        pickle.dump(CA_Provinces_result1, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    #CA_Provinces_result.to_file('../../Dash_app2/data/canada/CA Provinces.geojson' , driver='GeoJSON')
    CA_Provinces_result.to_file('../../Data/app updated data/CA Provinces.json' , driver='GeoJSON')
    with open('../../Data/app updated data/CA Provinces.json') as f:
        CA_Provinces_result_json = json.load(f)
    
    with open('../../Dash_app2/data/Canada regions location.pickle', 'wb') as handle:
        pickle.dump(CA_Provinces_result_json, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return(CA_Provinces_result)

In [25]:
CA_Provinces_result = CA_Provicnes(ON_regions, QC_regions, Alberta_regions, BC_regions, Manitoba_regions, 
                                   Saskatchewan_regions,New_Brunswick_regions, NL_regions, Nova_Scotia_regions, 
                                   Montreal_bor_df)
CA_Provinces_result.head()

,geometry,region,num cases,Province
0,"POLYGON ((-114.08376 51.29993, -114.07216 51.3...",City Of Airdrie,50.0,Alberta
1,"POLYGON ((-113.36156 55.02383, -113.05654 55.0...",Athabasca County,2.0,Alberta
2,"POLYGON ((-114.34920 54.50138, -114.30450 54.5...",County Of Barrhead No 11,2.0,Alberta
3,"POLYGON ((-113.46430 53.36665, -113.44263 53.3...",City Of Beaumont,9.0,Alberta
4,"POLYGON ((-113.05000 53.35949, -113.03879 53.3...",Beaver County,5.0,Alberta


In [26]:
CA_Provinces_result.sort_values(by = 'num cases', ascending = False).head(10)

,geometry,region,num cases,Province
89,"POLYGON ((-73.99869 45.44985, -73.89537 45.526...",Region de Montreal,27555.0,Quebec
133,"MULTIPOLYGON (((-79.35231 43.62732, -79.37091 ...",City of Toronto Health Unit,14777.0,Ontario
99,"POLYGON ((-72.31440 45.59707, -72.34360 45.598...",Region de la Monteregie,8117.0,Quebec
119,"POLYGON ((-79.63930 43.74985, -79.58876 43.664...",Peel Regional Health Unit,6564.0,Ontario
96,"POLYGON ((-73.89537 45.52658, -73.74508 45.668...",Region de Laval,5866.0,Quebec
7,"POLYGON ((-113.86554 51.05967, -113.86556 51.0...",City Of Calgary,4559.0,Alberta
97,"POLYGON ((-72.99682 46.11233, -73.04378 46.107...",Region de Lanaudiere,4289.0,Quebec
98,"POLYGON ((-73.74508 45.66848, -73.89537 45.526...",Region des Laurentides,3584.0,Quebec
131,"POLYGON ((-79.17025 43.85547, -79.37250 43.808...",York Regional Health Unit,3145.0,Ontario
173,"POLYGON ((-73.65430 45.58363, -73.65613 45.583...",Montreal Nord,2569.0,Montreal


In [27]:
CA_Provinces_result.loc[CA_Provinces_result['Province'] == 'Quebec']

,geometry,region,num cases,Province
84,"POLYGON ((-67.59801 47.99982, -68.12259 47.999...",Region du Bas Saint Laurent,62.0,Quebec
85,"POLYGON ((-69.87082 48.18161, -69.95675 48.057...",Region du Saguenay Lac Saint Jean,342.0,Quebec
86,"POLYGON ((-69.53634 48.05613, -69.61044 48.000...",Region de la Capitale Nationale,1874.0,Quebec
87,"POLYGON ((-74.44389 48.92928, -73.72845 48.470...",Region de la Mauricie et du Centre du Quebec,2053.0,Quebec
88,"POLYGON ((-70.41383 45.79610, -70.40022 45.720...",Region de l Estrie,979.0,Quebec
89,"POLYGON ((-73.99869 45.44985, -73.89537 45.526...",Region de Montreal,27555.0,Quebec
90,"POLYGON ((-74.80765 45.63842, -74.92831 45.645...",Region de l Outaouais,619.0,Quebec
91,"POLYGON ((-75.52046 47.84633, -75.72488 47.847...",Region de l Abitibi Temiscamingue,172.0,Quebec
92,"MULTIPOLYGON (((-67.49081 49.10157, -67.70522 ...",Region de la Cote Nord,123.0,Quebec
93,"POLYGON ((-70.02392 52.69636, -70.17528 52.594...",Region du Nord du Quebec,8.0,Quebec


### CTV

In [34]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
#options.add_argument('--headless')
options.add_argument('start-maximized')

browser = webdriver.Chrome(executable_path = '../chromedriver_win32/chromedriver.exe', options = options)

browser.get('https://www.ctvnews.ca/health/coronavirus/tracking-every-case-of-covid-19-in-canada-1.4852102#covid-top')

regions = ['Canada', 'British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 'Quebec', 'New Brunswick',
       'Nova Scotia', 'Prince Edward Island', 'Newfoundland and Labrador', 'Yukon', 'Northwest Territories',
       'Nunavut']
'''
#browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#for i in range(50,20000, 500):
#    browser.execute_script('window.scrollTo(0, '+str(i)+')')
#    time.sleep(0.1)
time.sleep(2)

for i in range(1, 14):
    browser.execute_script('window.scrollTo(0, 3500)')
    time.sleep(0.1)
    browser.find_element_by_xpath('//*[@id="covid-top"]/a['+str(i)+']').click()
    time.sleep(i/2)
    #browser.find_element_by_name('Top').click()
    browser.execute_script('window.scrollTo(0, 2000)')
'''

'\n#browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")\n#for i in range(50,20000, 500):\n#    browser.execute_script(\'window.scrollTo(0, \'+str(i)+\')\')\n#    time.sleep(0.1)\ntime.sleep(2)\n\nfor i in range(1, 14):\n    browser.execute_script(\'window.scrollTo(0, 3500)\')\n    time.sleep(0.1)\n    browser.find_element_by_xpath(\'//*[@id="covid-top"]/a[\'+str(i)+\']\').click()\n    time.sleep(i/2)\n    #browser.find_element_by_name(\'Top\').click()\n    browser.execute_script(\'window.scrollTo(0, 2000)\')\n'

In [48]:
import pickle

### Cases

Canada_cases = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[2]/table[1]/tbody/tr/td[1]').text.replace(',',''))
Total_cases = [Canada_cases]+[int(i.text.replace(',','').replace('aut','0')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[*]/div[2]/div[1]/table/tbody/tr/td[1]')]

Canada_active = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[2]/table[2]/tbody/tr/td[1]').text.replace(',',''))
Total_active = [Canada_active]+[int(i.text.replace(',','').replace('aut','0')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[*]/div[2]/div[2]/table/tbody/tr/td[1]')]

Canada_death = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[2]/table[2]/tbody/tr/td[2]').text.replace(',',''))
Total_death = [Canada_death]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[*]/div[2]/div[2]/table/tbody/tr/td[2]')]

Canada_recovered = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[2]/table[2]/tbody/tr/td[3]').text.replace(',',''))
Total_recovered = [Canada_recovered]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[*]/div[2]/div[2]/table/tbody/tr/td[3]')]

## New Cases
new_cases1 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div/div[2]/p[*]/strong')]
new_cases2 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/p[*]/b')]
new_cases3 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/div[*]/strong')]
new_cases4 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/strong')]
new_cases = new_cases1+new_cases2+new_cases3+new_cases4 
new_cases_filter = [i for i in new_cases if '(' in i]
new_cases_regions = [i.split(' (')[0] for i in new_cases_filter]
new_cases_num = [int(i.split(' (')[1].split(')')[0].replace(',','')) for i in new_cases_filter]
new_cases_dict = dict(zip(new_cases_regions, new_cases_num))

Country_summary = pd.DataFrame(columns = ['Province'])
Country_summary['Province'] = regions
Country_summary['Total Confirmed Cases'] = Total_cases
Country_summary['Total Active Cases'] = Total_active
Country_summary['Total Recovered Cases'] = Total_recovered
Country_summary['Total Deceased Cases'] = Total_death
Country_summary['Today Confirmed Cases'] = Country_summary['Province'].apply(lambda x : new_cases_dict[x] if x in new_cases_dict else 0)
Country_summary.loc[Country_summary['Province'] == 'Canada', 'Today Confirmed Cases'] = Country_summary.loc[1:,'Today Confirmed Cases'].sum()
Country_summary.sort_values(by = ['Total Confirmed Cases'], ascending = False, inplace = True)
#Country_summary.to_excel('../../Dash_app2/data/Canada_data_table.xlsx', index = False)
with open('../../Dash_app2/data/Canada_data_table.pickle', 'wb') as handle:
        pickle.dump(Country_summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
Canada_lat_lng = pd.read_excel('../../Data/app updated data/Canada_Provinces_lat_Lng.xlsx')
Country_summary_map = pd.merge(Canada_lat_lng, Country_summary[['Province','Total Confirmed Cases']], 
                              on = 'Province', how = 'left')
Country_summary_map['text'] = Country_summary_map .apply(lambda x : x['Province'] + '  :  ' + str('{:,}'.format(x['Total Confirmed Cases'])), axis = 1)
#Country_summary_map.to_excel('../../Dash_app2/data/Canada_map_data.xlsx', index = False)
with open('../../Dash_app2/data/Canada_map_data.pickle', 'wb') as handle:
        pickle.dump(Country_summary_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
def CTV_cases():
    import pandas as pd
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    options = Options()
    #options.add_argument('--headless')
    options.add_argument('start-maximized')

    browser = webdriver.Chrome(executable_path = '../chromedriver_win32/chromedriver.exe', options = options)

    browser.get('https://www.ctvnews.ca/health/coronavirus/tracking-every-case-of-covid-19-in-canada-1.4852102#covid-top')
    
    regions = ['Canada', 'British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba', 'Ontario', 'Quebec', 'New Brunswick',
           'Nova Scotia', 'Prince Edward Island', 'Newfoundland and Labrador', 'Yukon', 'Northwest Territories',
           'Nunavut']
    time.sleep(2)
    ### Cases
    
    Canada_cases = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/table[1]/tbody/tr/td[1]').text.replace(',',''))
    Total_cases = [Canada_cases]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[*]/div[*]/div[1]/table/tbody/tr/td[1]')]

    Canada_active = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/table[2]/tbody/tr/td[1]').text.replace(',',''))
    Total_active = [Canada_active]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[*]/div[*]/div[2]/table/tbody/tr/td[1]')]

    Canada_death = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/table[2]/tbody/tr/td[3]').text.replace(',',''))
    Total_death = [Canada_death]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[*]/div[*]/div[2]/table/tbody/tr/td[3]')]

    Canada_recovered = int(browser.find_element_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/table[2]/tbody/tr/td[2]').text.replace(',',''))
    Total_recovered = [Canada_recovered]+[int(i.text.replace(',','')) for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[*]/div[*]/div[2]/table/tbody/tr/td[2]')]

    ## New Cases
    new_cases1 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/p[*]/strong')]
    new_cases2 = [i.text for i in browser.find_elements_by_xpath('//*[@id="responsive_main"]/section/div/div/div[2]/div[1]/div[2]/p[*]/b')]
    new_cases = new_cases1+new_cases2
    new_cases_filter = [i for i in new_cases if '(' in i]
    new_cases_regions = [i.split(' (')[0] for i in new_cases_filter]
    new_cases_num = [int(i.split(' (')[1].split(')')[0].replace(',','')) for i in new_cases_filter]
    new_cases_dict = dict(zip(new_cases_regions, new_cases_num))

    Country_summary = pd.DataFrame(columns = ['Province'])
    Country_summary['Province'] = regions
    Country_summary['Total Confirmed Cases'] = Total_cases
    Country_summary['Total Active Cases'] = Total_active
    Country_summary['Total Recovered Cases'] = Total_recovered
    Country_summary['Total Deceased Cases'] = Total_death
    Country_summary['Today Confirmed Cases'] = Country_summary['Province'].apply(lambda x : new_cases_dict[x] if x in new_cases_dict else 0)
    Country_summary.loc[Country_summary['Province'] == 'Canada', 'Today Confirmed Cases'] = Country_summary.loc[1:,'Today Confirmed Cases'].sum()
    Country_summary.sort_values(by = ['Total Confirmed Cases'], ascending = False, inplace = True)
    Country_summary.to_excel('../../Dash_app2/data/Canada_data_table.xlsx', index = False)
    
    Canada_lat_lng = pd.read_excel('../../Data/app updated data/Canada_Provinces_lat_Lng.xlsx')
    Country_summary_map = pd.merge(Canada_lat_lng, Country_summary[['Province','Total Confirmed Cases']], 
                                  on = 'Province', how = 'left')
    Country_summary_map['text'] = Country_summary_map .apply(lambda x : x['Province'] + '  :  ' + str('{:,}'.format(x['Total Confirmed Cases'])), axis = 1)
    Country_summary_map.to_excel('../../Dash_app2/data/Canada_map_data.xlsx', index = False)
    return(Country_summary, Country_summary_map)

In [ ]:
#Country_summary, Country_summary_map = CTV_cases()

In [50]:
Country_summary

,Province,Total Confirmed Cases,Total Active Cases,Total Recovered Cases,Total Deceased Cases,Today Confirmed Cases
0,Canada,107346,27307,8773,71266,221
6,Quebec,56407,25014,5620,25773,91
5,Ontario,36594,1456,2716,32422,130
2,Alberta,8596,592,160,7844,0
1,British Columbia,3053,187,187,2679,0
8,Nova Scotia,1066,0,63,1000,0
3,Saskatchewan,815,43,15,757,0
4,Manitoba,325,4,7,314,0
10,Newfoundland and Labrador,262,1,3,258,0
7,New Brunswick,166,1,2,163,0


- US data

In [51]:
def US_Cases():
    import requests
    import pandas as pd
    import numpy as np
    
    url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'
    r = requests.get(url)

    with open('../../Data/app updated data/covid_confirmed_usafacts.csv', 'wb') as f:
        f.write(r.content)

    url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv'
    r = requests.get(url)

    with open('../../Data/app updated data/covid_deaths_usafacts.csv', 'wb') as f:
        f.write(r.content)
        
    US_Confirmed_df = pd.read_csv('../../Data/app updated data/covid_confirmed_usafacts.csv')
    US_Confirmed_df = US_Confirmed_df.loc[US_Confirmed_df['County Name'] != 'Statewide Unallocated'].reset_index(drop = True)
    US_Confirmed_df.loc[:,'countyFIPS'] = US_Confirmed_df.loc[:,'countyFIPS'].apply(lambda x : str("{:05d}".format(x)))
    for col in US_Confirmed_df.columns[4:]:
        US_Confirmed_df[col] = US_Confirmed_df[col].apply(lambda x : int(x))

    US_Confirmed_df['Last_7_days'] = US_Confirmed_df.iloc[:,-7:].mean(axis=1)
    US_Confirmed_df['Last_14_7_days'] = US_Confirmed_df.iloc[:,-14:-7].mean(axis=1)
    US_Confirmed_df['Total Cases'] = US_Confirmed_df.iloc[:,2:-2].sum(axis=1)
    for col in ['Last_7_days', 'Last_14_7_days', 'Total Cases']:
        US_Confirmed_df[col] = US_Confirmed_df[col].apply(lambda x : np.round(float(x),1))

    US_Confirmed_df['Weekly_dif'] = US_Confirmed_df.apply(lambda x : int(x['Last_7_days']- x['Last_14_7_days']) if 
                                                               x['Last_7_days'] > 5 and x['Last_14_7_days'] > 5 else 10000, axis = 1)
    US_Confirmed_df['Weekly_dif%'] = US_Confirmed_df.apply(lambda x : int((x['Last_7_days']- x['Last_14_7_days'])/(x['Last_14_7_days'])*100) if 
                                                                 x['Last_14_7_days']!=0  and x['Last_7_days'] > 5 and x['Last_14_7_days'] > 5 
                                                                 else (int((x['Last_7_days']- x['Last_14_7_days'])*100) if 
                                                                 x['Last_14_7_days']==0  and x['Last_7_days'] > 5 and x['Last_14_7_days'] > 5 else 10000)
                                                                 , axis = 1)
    US_Confirmed_df = US_Confirmed_df.loc[US_Confirmed_df['Weekly_dif%'] != 10000]
    US_Confirmed_df = US_Confirmed_df.reset_index(drop = True)

    cols = ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Last_7_days', 'Last_14_7_days', 'Total Cases', 
            'Weekly_dif', 'Weekly_dif%']
    US_Confirmed_df2 = US_Confirmed_df[cols]

    US_Death_df = pd.read_csv('../../Data/app updated data/covid_deaths_usafacts.csv')
    US_Death_df = US_Death_df.loc[:,['countyFIPS','County Name','State', US_Death_df.columns[-2]]]
    US_Death_df.rename(columns = {US_Death_df.columns[-1]:'Total Cases'}, inplace = True)
    US_Death_df = US_Death_df.loc[US_Death_df['County Name'] != 'Statewide Unallocated'].reset_index(drop = True)
    US_Death_df.loc[:,'countyFIPS'] = US_Death_df.loc[:,'countyFIPS'].apply(lambda x : str("{:05d}".format(x)))
    #US_Death_df.fillna(0, inplace = True)
    US_Death_df.loc[:,'Total Cases'] = US_Death_df.loc[:,'Total Cases'].apply(lambda x : int(x))
    #US_Death_df.to_excel('../../Dash_app2/data/US_Death.xlsx', index = False)
    US_Death_df.rename(columns = {'Total Cases':'Death'}, inplace = True)

    US_Confirmed_df3 = pd.merge(US_Confirmed_df2, US_Death_df, on = ['countyFIPS', 'County Name', 'State'], how = 'left')
    US_Confirmed_df3.fillna(value = {'Death':0}, inplace = True)
    with open('../../Dash_app2/data/US_Confirmed_Death.pickle', 'wb') as handle:
        pickle.dump(US_Confirmed_df3, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    #US_Confirmed_df3.to_excel('../../Dash_app2/data/US_Confirmed_Death.xlsx', index = False)
    return(US_Confirmed_df3)

In [52]:
US_Confirmed_Death = US_Cases()

In [53]:
US_Confirmed_Death.head()

,countyFIPS,County Name,State,stateFIPS,Last_7_days,Last_14_7_days,Total Cases,Weekly_dif,Weekly_dif%,Death
0,01001,Autauga County,AL,1,635.7,548.6,21651.0,87,15,14.0
1,01003,Baldwin County,AL,1,1003.7,722.9,31894.0,280,38,11.0
2,01005,Barbour County,AL,1,362.3,331.9,14019.0,30,9,2.0
3,01007,Bibb County,AL,1,201.3,175.6,7634.0,25,14,1.0
4,01009,Blount County,AL,1,258.7,214.3,8187.0,44,20,1.0
